In [40]:
import math
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
from typing import Tuple
from torch import optim

In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [43]:
patients = pd.read_csv("data/patients.csv").fillna(0)
encounters = pd.read_csv("data/admissions.csv").iloc[:, 1:].fillna(0)
lab_events = pd.read_csv("data/lab_events.csv").iloc[:, [0, 1, 2, 3]].fillna(0)
labels = pd.read_csv("data/labels.csv").fillna(0)
print(patients.shape)
print(encounters.shape)
print(lab_events.shape)
print(labels.shape)

(5000, 2)
(12013, 75)
(1671706, 4)
(12013, 3)


In [48]:
encounters.head()

,subject_id,hadm_id,admission_type_AMBULATORY OBSERVATION,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,...,race_PATIENT DECLINED TO ANSWER,race_PORTUGUESE,race_SOUTH AMERICAN,race_UNABLE TO OBTAIN,race_UNKNOWN,race_WHITE,race_WHITE - BRAZILIAN,race_WHITE - EASTERN EUROPEAN,race_WHITE - OTHER EUROPEAN,race_WHITE - RUSSIAN
0,10001843,21728396,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,10002751,22002850,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,10004764,24817563,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,10012688,23145708,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,10013300,23698883,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [45]:
num_lab_types = len(lab_events["itemid"].unique())
print(num_lab_types)

722


In [56]:
class PatientEncounter(Dataset):
    '''
    PatientEncounter Dataset class
    '''
    def __init__(self, patients, encounters, lab_events, labels):
        super().__init__()
        self.patients = patients
        self.encounters = encounters
        self.lab_events = lab_events
        self.labels = labels
        self.patient_ids = encounters.subject_id
        self.encounter_ids = encounters.hadm_id

    def __len__(self):
        return len(self.patient_ids)

    def __getitem__(self, index):
        patient_id = self.patient_ids[index]
        encounter_id = self.encounter_ids[index]
        
        # Load data for the given patient-encounter
        data_patient = torch.from_numpy(self.patients.loc[self.patients.subject_id == patient_id].iloc[:, 1:].astype(float).values).to(device)
        data_encounter = torch.from_numpy(self.encounters.loc[(self.encounters.subject_id == patient_id) & (self.encounters.hadm_id == encounter_id)].iloc[:, 2:].astype(float).values).to(device)
        data_lab_events = torch.from_numpy(self.lab_events.loc[(self.lab_events.subject_id == patient_id) & (self.lab_events.hadm_id == encounter_id)].iloc[:, 2:].astype(float).values).to(device)
        X = [data_patient, data_encounter, data_lab_events]
        y = torch.from_numpy(self.labels.loc[(self.labels.subject_id == patient_id) & (self.labels.hadm_id == encounter_id)].READMIT_ONE_WEEK.values).to(device)

        return X, y

class Patient(Dataset):
    '''
    Patient Dataset class
    '''
    def __init__(self, patients, labels):
        super().__init__()
        self.patients = patients
        self.labels = labels
        self.patient_ids = patients.subject_id

    def __len__(self):
        return len(self.patient_ids)

    def __getitem__(self, index):
        patient_id = self.patient_ids[index]
        
        # Load data for the given patient
        data_patient = torch.from_numpy(self.patients.loc[self.patients.subject_id == patient_id].values).to(device)
        X = [data_patient]
        y = torch.from_numpy(self.labels.loc[self.labels.subject_id == patient_id].READMIT_ONE_WEEK.values).to(device)

        return X, y

# Define a custom collate function to pad tensors in X to the same size
def collate_fn(batch):
    X, y = zip(*batch)
    X_patient = [x[0] for x in X]
    X_encounter = [x[1] for x in X]
    X_lab_events = [x[2] for x in X]
    X_lab_events_padded = pad_sequence(X_lab_events, batch_first=True, padding_value=0)
    X_lab_events_padded = [x.unsqueeze(0) for x in X_lab_events_padded]
    y = torch.cat(y, dim=0)
    return X_patient, X_encounter, X_lab_events_padded, y

train_data = PatientEncounter(patients, encounters, lab_events, labels)
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [6]:
example_x = next(iter(train_dataloader))
example_x[0][0].shape, example_x[1][0].shape, example_x[2][0].shape, example_x[3].shape

(torch.Size([1, 1]),
 torch.Size([1, 49]),
 torch.Size([1, 1044, 2]),
 torch.Size([64]))

In [51]:
class LabModule(nn.Module):
    '''
    Lab specific module, which takes in a batch of lab events and outputs a batch of lab embeddings.
    '''
    def __init__(self, num_lab_types):
        super().__init__()
        self.lab_type = nn.Linear(in_features=1, out_features=5) # 5-dim lab type embedding
        self.lab_value = nn.Linear(in_features=1, out_features=5) # 5-dim lab value embedding
        self.layer_out = nn.Linear(in_features=10, out_features=5) # 5-dim lab output embedding
    
    def forward(self, x):
        x = torch.cat(x, dim=0) # dim = (batch_size, max_seq_len, 2)
        x_type = x[:, :, 0].unsqueeze(2)
        x_value = x[:, :, 1].unsqueeze(2)
        out = torch.cat((self.lab_type(x_type), self.lab_value(x_value)), dim=2)
        out = self.layer_out(out) # dim = (batch_size, max_seq_len, 5)
        return(out)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1) # dim = (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)) # dim = (d_model/2)
        pe = torch.zeros(max_len, 1, d_model) # dim = (max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class PatientEncounterModel(nn.Module):
    '''
    Patient encounter DL model structure.
    '''
    def __init__(
        self,
        d_model: int, # embedding dimension
        nhead: int, # number of heads in multi-head attention
        d_hid: int, # dimension of feedforward network model
        nlayers: int, # number of encoder layers
        dropout: float, # dropout probability
        embed_event_dim: int, # dimension of event embedding
        num_static_features: int, # number of static features
        num_transformer_layers: 2, # number of transformer layers
        num_mlp_layers: int=2, # number of MLP layers
        num_mlp_features: int=11, # number of MLP features
        ):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Linear(embed_event_dim, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, embed_event_dim)
        self.lab_module = LabModule(num_lab_types=num_lab_types)
        self.layer_out = nn.Linear(in_features=num_static_features+d_model, out_features=10)
        self.fc1 = nn.Linear(in_features=10, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=10)
        self.fc3 = nn.Linear(in_features=10, out_features=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.lab_module(x[2]) # only lab-specific module for now
 
        # 4: feed modality-specific outputs into transformer architecture
        out = self.encoder(out) * math.sqrt(self.d_model)
        out = self.pos_encoder(out)
        out = self.transformer_encoder(out)
        out = torch.mean(out, dim=1, keepdim=True)

        # 5: concatenate outputs from transformer structure with static features
        out = torch.stack([out], dim=0).squeeze(0).squeeze(1)
        x_patient = torch.stack(x[0], dim=0).squeeze(2)
        x_encounter = torch.stack(x[1], dim=0).squeeze(1)
        out = torch.cat([x_patient, x_encounter, out], dim=1)

        # 6: feed concatenated output into MLP architecture
        out = self.layer_out(out)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = self.sigmoid(out)
        out = out.squeeze(1)

        return(out)

model = PatientEncounterModel(
    num_static_features=74, # number of static features
    num_transformer_layers=2, # number of transformer layers
    num_mlp_layers=2, # number of MLP layers
    num_mlp_features=11, # number of MLP features
    d_model=100, # embedding dimension
    nhead=2, # number of heads in multi-head attention
    d_hid=5, # dimension of feedforward network model
    nlayers=2, # number of encoder layers
    dropout=0.1, # dropout probability
    embed_event_dim=5 # dimension of event embedding
).to(device).double()

x = next(iter(train_dataloader))
model(x)

tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000], dtype=torch.float64,
       grad_fn=<SqueezeBackward1>)

In [63]:
# define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.000001, momentum=0.9)

In [64]:
running_loss = 0.0
for epoch in range(5):  # loop over the dataset multiple times
    for i, data in enumerate(train_dataloader):
        # get the inputs; data is a list of [patient, encounter, lab, y]
        X_patient, X_encounter, X_lab, y = data
        X_patient = [x.to(device) for x in X_patient]
        X_encounter = [x.to(device) for x in X_encounter]
        X_lab = [x.to(device) for x in X_lab]
        y = y.to(device).double()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model([X_patient, X_encounter, X_lab])
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 4: # print every 5 mini-batches
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0

[1,     5] loss: 0.693
[1,    10] loss: 0.693
[1,    15] loss: 0.693
[1,    20] loss: 0.693
[1,    25] loss: 0.693
[1,    30] loss: 0.693
[1,    35] loss: 0.693
[1,    40] loss: 0.693
[1,    45] loss: 0.693
[1,    50] loss: 0.693
[1,    55] loss: 0.693
[1,    60] loss: 0.693


[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [36]:
len(train_dataloader)

27